In [2]:
import gym
import numpy as np 
from helpers import *
from ddpg import * 
import sys
import numpy as np

In [3]:
# Set up
env = gym.make("Pendulum-v1")
norm_env = NormalizedEnv(env)

agent = DDPGAgent(norm_env, 1e-4, 1e-4, 0.99, 100000, 0.3, 256)

critic_losses = []
actor_losses = []
rewards = []
batch_size = 128

In [ ]:
for episode in range(50):
    state, info = norm_env.reset()
    episode_reward = 0
    episode_critic_loss = 0
    episode_actor_loss = 0
    trunc = False
    
    while not trunc:
        action = agent.compute_action(state)
        next_state, reward, terminated, trunc, info = norm_env.step(action) 
        agent.buffer.add(state, action, reward, next_state, trunc)
        
        if len(agent.buffer) > batch_size:
            transition = agent.buffer.sample(batch_size)
            aloss, closs, reward = agent.update(transition)
            #print("reward :", reward)
            episode_critic_loss += closs
            episode_actor_loss += aloss
        
        state = next_state
        episode_reward = torch.mean(reward)
        #print("type of reward :", type(episode_reward))

        if trunc:
            sys.stdout.write("episode: {}, reward: {}, critic loss: {}, actor loss: {} \n".format(episode, episode_reward, episode_critic_loss, episode_actor_loss))
            break

    rewards.append(episode_reward)
    critic_losses.append(episode_critic_loss)
    actor_losses.append(episode_actor_loss)